https://github.com/tensorflow/tensorflow

```bash
# For Mac CPU only: Python 2
wget http://ci.tensorflow.org/view/Nightly/job/nightly-matrix-cpu/TF_BUILD_CONTAINER_TYPE=CPU,TF_BUILD_IS_OPT=OPT,TF_BUILD_IS_PIP=PIP,TF_BUILD_PYTHON_VERSION=PYTHON2,label=mac-slave/lastSuccessfulBuild/artifact/pip_test/whl/tensorflow-0.8.0-py2-none-any.whl
pip install ./tensorflow-0.8.0-py2-none-any.whl
```

In [68]:
import numpy as np
import tensorflow as tf

In [69]:
hello = tf.constant('Hello, TensorFlow!')

In [70]:
sess = tf.Session()
sess.run(hello)

'Hello, TensorFlow!'

In [71]:
a, b = tf.constant(10), tf.constant(32)
sess.run(a+b)

42

In [72]:
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

In [73]:
binary_encode(128, 10)

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [74]:
def fizz_buzz_encode(i):
    if   i % 15 == 0: return np.array([0, 0, 0, 1])
    elif i % 5  == 0: return np.array([0, 0, 1, 0])
    elif i % 3  == 0: return np.array([0, 1, 0, 0])
    else:             return np.array([1, 0, 0, 0])

In [75]:
NUM_DIGITS = 10
trX = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = np.array([fizz_buzz_encode(i)          for i in range(101, 2 ** NUM_DIGITS)])

In [76]:
(trX, trY)

(array([[1, 0, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ..., 
        [1, 0, 1, ..., 1, 1, 1],
        [0, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]]), array([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        ..., 
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0]]))

In [77]:
NUM_HIDDEN = 10000

In [78]:
X = tf.placeholder("float", [None, NUM_DIGITS])
Y = tf.placeholder("float", [None, 4])

In [79]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

w_h = init_weights([NUM_DIGITS, NUM_HIDDEN])
w_o = init_weights([NUM_HIDDEN, 4])

In [80]:
def model(X, w_h, w_o):
    h = tf.nn.relu(tf.matmul(X, w_h))
    return tf.matmul(h, w_o)

In [81]:
py_x = model(X, w_h, w_o)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost)

In [82]:
predict_op = tf.argmax(py_x, 1)

In [83]:
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

In [84]:
BATCH_SIZE = 128
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    for epoch in range(10000):
        p = np.random.permutation(range(len(trX)))
        trX, trY = trX[p], trY[p]
        for start in range(0, len(trX), BATCH_SIZE):
            end = start + BATCH_SIZE
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
        if epoch % 1000 == 0:
            print(epoch, np.mean(np.argmax(trY, axis=1) ==
                             sess.run(predict_op, feed_dict={X: trX, Y: trY})))
    numbers = np.arange(1, 101)
    teX = np.transpose(binary_encode(numbers, NUM_DIGITS))
    teY = sess.run(predict_op, feed_dict={X: teX})
    output = np.vectorize(fizz_buzz)(numbers, teY)

    print(output)

(0, 0.53412784398699886)
(1000, 0.96749729144095342)
(2000, 1.0)
(3000, 1.0)
(4000, 1.0)
(5000, 1.0)
(6000, 1.0)
(7000, 1.0)
(8000, 1.0)
(9000, 1.0)
['1' '2' 'fizz' '4' 'buzz' 'fizz' '7' '8' 'fizz' 'buzz' '11' 'fizz' '13'
 '14' 'fizzbuzz' '16' '17' 'fizz' '19' 'buzz' 'fizz' '22' '23' 'fizz'
 'buzz' '26' 'fizz' '28' '29' 'fizzbuzz' '31' '32' 'fizz' '34' 'buzz'
 'fizz' '37' '38' 'fizz' 'buzz' '41' 'fizz' '43' '44' 'fizzbuzz' '46' '47'
 'fizz' '49' 'buzz' 'fizz' '52' '53' 'fizz' 'buzz' '56' 'fizz' '58' '59'
 'fizzbuzz' '61' '62' 'fizz' '64' 'buzz' 'fizz' '67' '68' 'fizz' 'buzz'
 '71' 'fizz' '73' '74' 'fizzbuzz' '76' '77' 'fizz' '79' 'buzz' 'fizz' '82'
 '83' 'fizz' 'buzz' '86' 'fizz' '88' '89' 'fizzbuzz' '91' '92' 'fizz' '94'
 'buzz' 'fizz' '97' '98' 'fizz' 'buzz']


In [88]:
def direct_fizz_buzz(i):
    if   i % 15 == 0: return 'fizzbuzz'
    elif i % 5  == 0: return 'buzz'
    elif i % 3  == 0: return 'fizz'
    else:             return str(i)

In [89]:
correct_fizz_buzz = [direct_fizz_buzz(i) for i in range(1, 100)]

In [90]:
for i in range(0, 99):
    if output[i] != correct_fizz_buzz[i]:
        print i+1, output[i], correct_fizz_buzz[i]

https://www.tensorflow.org/versions/r0.8/tutorials/mnist/beginners/index.html

In [91]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [94]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
for i in range(100000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [93]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9121


In [95]:
scores = [3.0, 1.0, 0.2]
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    expx = np.exp(x)
    return expx / np.sum(expx, axis=0)
print(softmax(scores))

[ 0.8360188   0.11314284  0.05083836]


In [101]:
num = 1000000000
for i in range(0, 1000000):
    num += 0.000001
round(num - 1000000000, 3)

0.954